## Tutoriel : interagir avec le système de stockage S3 du SSP Cloud (MinIO)

In [10]:
import os

import pandas as pd
import s3fs
import zipfile

### Récupérer les données d'un challenge

In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
# Lister les challenges
fs.ls("gvimont/diffusion/hackathon-minarm-2024")

['gvimont/diffusion/hackathon-minarm-2024/AIVSAI',
 'gvimont/diffusion/hackathon-minarm-2024/Acoustique',
 'gvimont/diffusion/hackathon-minarm-2024/Similarité']

In [5]:
# Lister les fichiers d'un challenge
fs.ls("gvimont/diffusion/hackathon-minarm-2024/AIVSAI")

['gvimont/diffusion/hackathon-minarm-2024/AIVSAI/.keep',
 'gvimont/diffusion/hackathon-minarm-2024/AIVSAI/HC3.zip']

In [6]:
# Télécharger les données dans le service
PATH_IN = 'gvimont/diffusion/hackathon-minarm-2024/AIVSAI/HC3.zip'
fs.download(PATH_IN, 'data/HC3.zip')

[None]

In [7]:
# Décompresser les données
with zipfile.ZipFile("data/HC3.zip","r") as zip_file:
    zip_file.extractall("data/")

NB : les données peuvent être également téléchargées directement si besoin, pour être utilisées hors du SSP CLoud.
Exemple pour le fichier ci-dessus (même format de lien pour les autres challenges) : 

http://minio.lab.sspcloud.fr/gvimont/diffusion/hackathon-minarm-2024/AIVSAI/HC3.zip

### Exporter des données

In [12]:
df = pd.read_json('data/HC3/medicine.jsonl', lines=True)

In [14]:
df.head()

,question,human_answers,chatgpt_answers
0,Does Primolut N taken during pregnancy affect ...,"[Hi, Thanks for the query. I understand yo...",[It is not recommended to use Primolut N durin...
1,Bloating and pain on right lower abdomen. Shou...,"[Hello,Thanks for the query to H.C.M. Forum.Pa...",[If you are experiencing abdominal pain and bl...
2,Is chest pain related to intake of clindamycin...,"[Hello, The use of Clindamycin can cause stoma...",[It is possible that chest pain could be relat...
3,Q. Noticed a yellowish sag in the gums of my 1...,[Hello. Revert back with the photos to a denti...,[It is difficult to accurately diagnose a cond...
4,"Suggest remedy for low grade fever, hot and co...","[Hi Dear,Welcome to Healthcaremagic Team.Under...",[I'm sorry to hear that you're feeling sick. I...


In [19]:
# Export vers un bucket personnel
PATH_OUT = 'avouacr/diffusion/projet-mongroupe-hackathon/medicine.csv'

with fs.open(PATH_OUT, 'w') as file_out:
    df.to_csv(file_out, index=False)

In [20]:
# NB : le dossier 'diffusion' permet un accès en lecture à tous les membres du groupe !
# Tous les membres peuvent donc le voir et l'utiliser dans un service
fs.ls("avouacr/diffusion/projet-mongroupe-hackathon")

['avouacr/diffusion/projet-mongroupe-hackathon/medicine.csv']

In [21]:
with fs.open(PATH_OUT, mode="r") as file_in:
    df_test = pd.read_csv(file_in)

In [22]:
df_test.head()

,question,human_answers,chatgpt_answers
0,Does Primolut N taken during pregnancy affect ...,"['Hi, Thanks for the query. I understand y...",['It is not recommended to use Primolut N duri...
1,Bloating and pain on right lower abdomen. Shou...,"['Hello,Thanks for the query to H.C.M. Forum.P...",['If you are experiencing abdominal pain and b...
2,Is chest pain related to intake of clindamycin...,"['Hello, The use of Clindamycin can cause stom...",['It is possible that chest pain could be rela...
3,Q. Noticed a yellowish sag in the gums of my 1...,['Hello. Revert back with the photos to a dent...,"[""It is difficult to accurately diagnose a con..."
4,"Suggest remedy for low grade fever, hot and co...","['Hi Dear,Welcome to Healthcaremagic Team.Unde...","[""I'm sorry to hear that you're feeling sick. ..."
